In [1]:
!pip install -q langchain langchain_core langchain_community sentence_transformers faiss-cpu unstructured chromadb Cython tiktoken unstructured[local-inference] langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (2,078 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126111 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...


In [21]:
import os

from langchain.document_loaders import UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [5]:
os.environ["GROQ_API_KEY"] = "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"

In [8]:
import requests
urls = "https://web.stanford.edu/class/archive/cs/cs106ap/cs106ap.1198/lectures/6-PythonFunctions/6-Python_Functions.pdf"
response = requests.get(urls)

In [9]:
with open("6-Python_Functions.pdf", "wb") as f:
  f.write(response.content)

In [10]:
loader  = UnstructuredFileLoader("6-Python_Functions.pdf")

<ipython-input-10-2328390741>:1: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader  = UnstructuredFileLoader("6-Python_Functions.pdf")


In [11]:
documents = loader.load()
documents

[Document(metadata={'source': '6-Python_Functions.pdf'}, page_content='Python Functions CS106AP Lecture 6\n\nRoadmap\n\nDay 1!\n\nGraphics\n\nProgramming Basics\n\nMidterm\n\nObject-Oriented Programming\n\nThe Console\n\nImages\n\nData structures\n\nEveryday Python\n\nLife after CS106AP!\n\nRoadmap\n\nDay 1!\n\nGraphics\n\nProgramming Basics\n\nThe Console\n\nImages\n\nPython Functions\n\nStrings and the Console\n\nData structures\n\nMidterm\n\nObject-Oriented Programming\n\nEveryday Python\n\nLife after CS106AP!\n\nToday’s questions\n\nHow do we translate what we know from Karel into regular Python code?\n\nHow can we make our code more ﬂexible by producing diﬀerent outputs depending on the input?\n\nToday’s topics\n\n1.\n\nIntroduction and Review\n\n2. Range For Loops\n\n3. Python Functions\n\n4. Variable Scope\n\n5. What’s next?\n\nWho am I?\n\nSonja Johnson-Yu\n\nSonja Johnson-Yu\n\nSonja Johnson-Yu\n\nSonja Johnson-Yu\n\nReview\n\nVariables\n\nWhat is a variable?\n\nA variable is 

In [13]:
textsplitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400)

In [15]:
texts = textsplitter.split_documents(documents)

In [16]:
type(texts)

list

In [17]:
len(texts)

8

In [18]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-18-3655315981>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-18-3655315981>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingfac

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
persist_directory = "doc_db"


In [22]:
vector_db = Chroma.from_documents(
    documents = texts,
    embedding = embeddings,
    persist_directory = persist_directory,
)

In [23]:
retriever = vector_db.as_retriever()

In [27]:
llm = ChatGroq(
    model = "llama-3.3-70b-versatile",
    temperature = 0
)

In [28]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    )

In [29]:
query = "Anatomy of a Function"
response = qa_chain.invoke({"query":query})

In [30]:
print(response)

{'query': 'Anatomy of a Function', 'result': 'The anatomy of a function consists of the following components:\n\n1. **Function Definition**: This is the declaration of the function, which includes the function name, parameters, and the code that the function will execute.\n2. **Function Name**: This is the name given to the function, which is used to identify and call the function.\n3. **Parameters**: These are the variables that a function expects as input. They are also known as arguments.\n4. **Return Value**: This is the value that a function hands back to the "calling" function after it has finished executing.\n\nIn the given code example:\n```\ndef average(a, b):\n    sum = a + b\n    return sum / 2\n```\n* The **function definition** is `def average(a, b): ...`.\n* The **function name** is `average`.\n* The **parameters** are `a` and `b`.\n* The **return value** is `sum / 2`.', 'source_documents': [Document(metadata={'source': '6-Python_Functions.pdf'}, page_content='parameters\